In [1]:
import pandas as pd 
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [2]:
tips= sns.load_dataset("tips")
tips

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2


In [3]:
tips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [13]:
y= tips["tip"]
x= pd.get_dummies(tips.drop("tip", axis= "columns"), columns= ["sex", "smoker", "day", "time"], drop_first=True)
x

,total_bill,size,sex_Female,smoker_No,day_Fri,day_Sat,day_Sun,time_Dinner
0,16.99,2,1,1,0,0,1,1
1,10.34,3,0,1,0,0,1,1
2,21.01,3,0,1,0,0,1,1
3,23.68,2,0,1,0,0,1,1
4,24.59,4,1,1,0,0,1,1
...,...,...,...,...,...,...,...,...
239,29.03,3,0,1,0,1,0,1
240,27.18,2,1,0,0,1,0,1
241,22.67,2,0,0,0,1,0,1
242,17.82,2,0,1,0,1,0,1


In [14]:
X = x[["total_bill", "size", "time_Dinner"]]
X

,total_bill,size,time_Dinner
0,16.99,2,1
1,10.34,3,1
2,21.01,3,1
3,23.68,2,1
4,24.59,4,1
...,...,...,...
239,29.03,3,1
240,27.18,2,1
241,22.67,2,1
242,17.82,2,1


In [6]:
y

0      1.01
1      1.66
2      3.50
3      3.31
4      3.61
       ... 
239    5.92
240    2.00
241    2.00
242    1.75
243    3.00
Name: tip, Length: 244, dtype: float64

In [7]:
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size= 0.3, random_state= 42)

In [8]:
print(f"{X_train.shape= }")
print(f"{y_train.shape= }")
print(f"{X_test.shape= }")
print(f"{y_test.shape= }")

X_train.shape= (170, 3)
y_train.shape= (170,)
X_test.shape= (74, 3)
y_test.shape= (74,)


In [9]:
from sklearn.preprocessing import StandardScaler

scaler= StandardScaler()
scaled_X_train= scaler.fit_transform(X_train)
scaled_X_test= scaler.transform(X_test)

scaled_X_train.mean(), scaled_X_train.std(), scaled_X_test.mean(), scaled_X_test.std()

(-2.7864421010200007e-17, 1.0, -0.05578631087270823, 0.9902814291133639)

In [10]:
from sklearn.preprocessing import PolynomialFeatures

model_polynomial= PolynomialFeatures(2, include_bias= False)

#poly_features= model_polynomial.fit(scaled_X_train)
X_train_direct= model_polynomial.fit_transform(scaled_X_train)

X_train_poly= model_polynomial.transform(scaled_X_train)
X_test_poly= model_polynomial.transform(scaled_X_test)

np.allclose(X_train_poly, X_train_direct)
X_train_poly.shape

(170, 9)

In [12]:
from sklearn.linear_model import LinearRegression


test_RAMSE = []
train_RAMSE = []
for degree in range(1, 5):
    poly = PolynomialFeatures(degree=degree, include_bias=False)

    poly_X_train = poly.fit_transform(scaled_X_train)
    poly_X_test = poly.fit_transform(scaled_X_test)

    print(poly_X_test.shape)
    model_linear_regression = LinearRegression()
    model_linear_regression.fit(poly_X_train, y_train)

    y_train_pred = model_linear_regression.predict(poly_X_train)
    y_test_pred = model_linear_regression.predict(poly_X_test)

    train_RAMSE.append(np.sqrt(mean_squared_error(y_train, y_train_pred)))
    test_RAMSE.append(np.sqrt(mean_squared_error(y_test, y_test_pred)))


test_RAMSE, train_RAMSE

(74, 3)
(74, 9)
(74, 19)
(74, 34)


([0.9251663999141493,
  1.0451350706430025,
  1.204596900719222,
  2.0408446651417274],
 [1.0506868081153957,
  1.029109990344908,
  0.9529579879823995,
  0.8442316172563938])